# Experiment with Fashion MNIST
## Federated learning with ours 3

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTHGRAPH_VERBOSITY'] = '0'
import pickle
import time
import datetime
import random
import operator
import copy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.random.set_seed(6292)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
today

In [ ]:
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-fashionmnist'))
dataset_root

In [ ]:
model_root = os.path.abspath(os.path.expanduser(f'model-fashionmnist-ours3-{today}'))
model_root

In [ ]:
found = pd.read_csv('round-fashionmnist.csv', index_col='Clients')
found.fillna(-1, inplace=True)
found = found.astype('int')
found.drop(columns=['Total'], inplace=True)
found.drop(index=['Total'], inplace=True)
found

In [ ]:
found_round = np.unique(found.values).tolist()
found_round.remove(-1)
found_round.sort(reverse=False)
found_round

In [ ]:
found_label = dict()
for fr in found_round:
    label = found_label.get('all', dict())
    label[fr] = found.columns[((-1 < found.loc[:, :]) & 
                               (found.loc[:, :] <= fr)).any()].tolist()
    found_label['all'] = label
    for c in found.index:
        label = found_label.get(c, dict())
        label[fr] = found.columns[((-1 < found.loc[c, :]) & 
                                   (found.loc[c, :] <= fr))].tolist()
        found_label[c] = label
found_label

In [ ]:
found_client = dict()
for fr in found_round:
    found_client[fr] = found.index[((-1 < found.loc[:, :]) & 
                                    (found.loc[:, :] <= fr)).any(axis=1)].tolist()
found_client

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
target_size = (28, 28, 3)

In [ ]:
model = tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=target_size),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(128, activation='relu'),
                       tf.keras.layers.Dense(len(found_label['all'][0]))])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_path = os.path.join(model_root, f'{0}', 'global')
model.save(model_path)

In [ ]:
max_round = 200
cil_round = 10
max_select = 5
epochs = 10
results = []
for fr in range(max_round+1):
    # Handle CIL
    cil = 0
    if fr in found_round:
        round_client = found_client[fr]
        round_label = found_label['all'][fr]
        ### model
        global_model_path = os.path.join(model_root, f'{fr}', 'global')
        model = tf.keras.models.load_model(global_model_path)
        new_model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=target_size),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(len(round_label))])
        new_model.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])
        weights = model.get_weights()
        diff = int(abs(weights[-1].shape[0]-len(found_label['all'][fr])))
        if diff != 0:
            weights[-2] = np.pad(weights[-2], (0, diff), mode='constant', constant_values=0)[:-diff,:]
            weights[-1] = np.pad(weights[-1], (0, diff), mode='constant', constant_values=0)
            for weight in weights[-2]:
                for idx in range(-diff, 0):
                    weight[idx] = random.uniform(-1, 1)
        new_model.set_weights(weights)
        for layer in new_model.layers[:-2]:
            layer.trainable = False
        model_path = os.path.join(model_root, f'{fr}', 'global')
        new_model.save(model_path)
        cil = cil_round
        ### datasets
        datasets = dict()
        dataset_path = os.path.join(dataset_root, 'center', 'test')
        datasets['all'] = image_generator.flow_from_directory(dataset_path, classes=round_label,
                                                              target_size=target_size[:-1], shuffle=True,
                                                              follow_links=True)
        for client in round_client:
            train_dataset_path = os.path.join(dataset_root, 'scenario', f'{client}', f'{fr}', 'train')
            test_dataset_path = os.path.join(dataset_root, 'scenario', f'{client}', f'{fr}', 'test')
            datasets[client] = (image_generator.flow_from_directory(train_dataset_path, classes=round_label,
                                                                    target_size=target_size[:-1], shuffle=True,
                                                                    follow_links=True),
                                image_generator.flow_from_directory(test_dataset_path, classes=round_label,
                                                                    target_size=target_size[:-1], shuffle=True,
                                                                    follow_links=True))
    if cil > 0:
        cil = cil - 1
    # Ready for global model and client model
    global_model_path = os.path.join(model_root, f'{fr}', 'global')
    knowledges = []
    for client in round_client:
        model = tf.keras.models.load_model(global_model_path)
        if cil == 0:
            for layer in model.layers:
                layer.trainable = True
        model.fit(datasets[client][0], 
                  # validation_data=datasets[client][1],
                  epochs=epochs, verbose=0)
        model_path = os.path.join(model_root, f'{fr}', f'{client}')
        model.save(model_path)
        knowledges.append(model.get_weights())
    aggregates = copy.deepcopy(knowledges[0])
    for knowledge in knowledges[1:]:
        for i in range(len(knowledge)):
            aggregates[i] = aggregates[i] + knowledge[i]
    for i in range(len(aggregates)):
        aggregates[i] = aggregates[i] / len(knowledges)
    global_model = tf.keras.models.load_model(global_model_path)
    global_model.set_weights(aggregates)
    # Raw
    confusions = dict()
    for client in round_client:
        actuals = datasets[client][1].classes
        predictions = np.argmax(global_model.predict(datasets[client][1], verbose=0), axis=1)
        confusions[client] = tf.math.confusion_matrix(actuals, predictions, len(round_label)).numpy()
    # accuracy per each class
    accuracies = dict()
    for client in round_client:
        acc = []
        for l in range(len(round_label)):
            sample_num = sum(confusions[client][l])
            if sample_num != 0:
                acc.append(confusions[client][l][l] / sample_num)
            else:
                acc.append(-1)
        accuracies[client] = acc
    # client per each class
    parties = dict()
    for client in round_client:
        for l in range(len(round_label)):
            if accuracies[client][l] != -1:
                parties[l] = parties.get(l, 0) + 1
    # raw effects by classes
    effects_class = dict()
    for l in range(len(round_label)):
        acc_sum = 0
        for client in round_client:
            acc = accuracies[client][l]
            if acc != -1:
                acc_sum = acc_sum + acc
        effect = dict()
        for client in round_client:
            acc = accuracies[client][l]
            if acc != -1 and acc_sum != 0:
                effect[client] = acc / acc_sum
            elif acc == 0 and acc_sum == 0:
                effect[client] = 1 / parties[l]
            else:
                effect[client] = -1
        effects_class[l] = effect
    # effects per clients
    divs = dict()
    for l in range(len(round_label)):
        divs[l] = 1 / len(round_label)
    effects = dict()
    for key, value in effects_class.items():
        for client, effect in value.items():
            if effect != -1:
                effects[client] = effects.get(client, 0) + effect*divs[key]
    # selection with contribution
    effect_list = list(effects.items())
    effect_list.sort(key=operator.itemgetter(1), reverse=True)
    knowledges = []
    selected = set()
    for client, contrib in effect_list[:min(max_select, len(effect_list))]:
        selected.add(client)
        model_path = os.path.join(model_root, f'{fr}', f'{client}')
        model = tf.keras.models.load_model(model_path)
        knowledges.append(model.get_weights())
    aggregates = copy.deepcopy(knowledges[0])
    for knowledge in knowledges[1:]:
        for i in range(len(knowledge)):
            aggregates[i] = aggregates[i] + knowledge[i]
    for i in range(len(aggregates)):
        aggregates[i] = aggregates[i] / len(knowledges)
    global_model_path = os.path.join(model_root, f'{fr}', 'global')
    model = tf.keras.models.load_model(global_model_path)
    model.set_weights(aggregates)
    result = model.evaluate(datasets['all'], verbose=0)
    results.append(result)
    print(f'Federated round: {fr+1}, Result: {result}, Selected: {selected} with {effect_list}')
    global_model_path = os.path.join(model_root, f'{fr+1}', 'global')
    model.save(global_model_path)

In [ ]:
with open(f'result-fashionmnist-federatedlearning-ours3-{today}.pkl', 'wb') as f:
    pickle.dump(results, f)